In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table as dt
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter, animals



###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "Missy3434"

# Connect to database via CRUD Module
db = AnimalShelter

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(animals.readAll({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
#df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))
app.layout = html.Div([
    html.Center(html.B(html.H1('Richard Howell Project 2'))),
    html.Hr(),
    html.Img(
        src='data:image/png;base64,{}'.format(encoded_image.decode()),
        style={'height':'15%','width':'15%'}
        ),
    html.Div(
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Reset', 'value': 'RESET'},
                {'label': 'Water Rescue', 'value': 'WATER'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'MOUNTAIN OR WILDERNESS'},
                {'label': 'Disaster or Individual Tracking', 'value': 'DISASTER OR INDIVIDUAL TRACKING'}
            ]
        )

    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10
        
        
    ),
    html.Br(),
    html.Hr(),
     #This sets up the dashboard so that the dropdown menu, pie chart, and geolocation map are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
                html.Div([
                # Dropdown menu for pie chart
                    dcc.Dropdown(
                        id='pie_dropdown',
                        className='col s12 m6',
                        options=[
                            {'label': 'Breed', 'value': 'breed'},
                            {'label': 'Age', 'value': 'age_upon_outcome'},       
                            {'label': 'Outcome Type', 'value': 'outcome_type'}
                        ],
                        value='breed',
                        multi=False,
                        clearable=False
                        )
                    ],
                style={"width":"10%"}
                ),
                # Pie chart
                html.Div(
                    id='graph-id',
                    className='col s12 m6',
                    style={"width":"45%"}
                    
                ),
                # Geolocation map
                html.Div(
                    id='map-id',
                    className='col s12 m6',
                    style={"width":"45%"}
                    )
                ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# This callback filters the data supplied to the dataframe based on user selected radial button    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns'),
               Output('datatable-id','selected_rows')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    # Code to filter interactive data table with MongoDB queries
    # Query database for characteristics specified by client for each filter type
    # Additionally, exclude euthanized animals
    if filter_type == 'RESET':
        df = pd.DataFrame.from_records(animals.readAll({}))
    elif filter_type == 'WATER':
        df = pd.DataFrame(list(animals.readAll({
            "animal_type":"Dog",
            "breed":{"$in":["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome":"Intact Female",
            "age_upon_outcome_in_weeks":{"$gte": 26},
            "age_upon_outcome_in_weeks":{"$lte": 156},
            "outcome_type":{"$nin":["Euthanasia"]}
        })))
    elif filter_type == 'MOUNTAIN OR WILDERNESS':
        df = pd.DataFrame(list(animals.readAll({
            "animal_type":"Dog",
            "breed":{"$in":["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", 
                          "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome":"Intact Male",
            "age_upon_outcome_in_weeks":{"$gte": 26},
            "age_upon_outcome_in_weeks":{"$lte": 156},
            "outcome_type":{"$nin":["Euthanasia"]}
        })))
    elif filter_type == 'DISASTER OR INDIVIDUAL TRACKING':
        df = pd.DataFrame(list(animals.readAll({
            "animal_type":"Dog",
            "breed":{"$in":["Doberman Pinscher", "German Shepherd", "Golden Retriever", 
                          "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome":"Intact Male",
            "age_upon_outcome_in_weeks":{"$gte": 20},
            "age_upon_outcome_in_weeks":{"$lte": 300},
            "outcome_type":{"$nin":["Euthanasia"]}
        })))
    
    else:
        df = pd.DataFrame.from_records(animals.readAll({}))
        
    # Prepare data and columns to be sent to data table
    data=df.to_dict('records')
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    # Reset selected row to first row when filter setting changes
    selected_rows = [0]
    
    return (data, columns, selected_rows)



# This callback highlights a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_rows')]
)
def update_styles(selectedRows):
    return [{
        'if': { 'row_index': i },
        'background_color': '#02F300'
    } for i in selectedRows]


# This callback populates the pie chart based on the user's dropdown selection.
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('pie_dropdown', 'value')])
def update_graphs(viewData, dropdownValue):
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(            
            figure = px.pie(
                data_frame=dff, 
                names=dropdownValue)
        )    
    ]


# This callback populates the geolocation map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', 'selected_rows')])
def update_map(viewData, selectedRows):   
    
    # Get selected row, or default to first row if no selection
    if not selectedRows:
        selected_row = 0
    else:   
        selected_row = selectedRows[0]
        
    # Get data from datatable                           
    dff = pd.DataFrame.from_dict(viewData)   
    
    # Get coordinate position data from selected row
    posX = dff.iloc[selected_row, 13]
    posY = dff.iloc[selected_row, 14]
    
    # Get name from selected row, replace empty strings with "No Name" for clarity
    name = dff.iloc[selected_row,9] 
    if name == "":
        name = "No Name"
        
    # Return a map using data pulled from selected row
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[posX,posY], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[posX,posY], children=[
                dl.Tooltip(dff.iloc[selected_row,4]),
                dl.Popup([
                    html.P("Animal Name"),
                    html.H1(name)
                ])
            ])
        ])
    ]                           


    
app.run_server(debug=True)

Dash app running on http://127.0.0.1:20034/
